# How to generate a combined utility/privacy report?

### Create a combined report of the metrics, whether they are utility or privacy metrics. /!\ Only for the summary.

Assume that the synthetic data is already generated \
Based on the Wisconsin Breast Cancer Dataset (WBCD)

In [1]:
# Standard library
import sys
import tempfile

sys.path.append("..")

# 3rd party packages
import pandas as pd

# Local packages
import config
import utils.draw
from metrics.report import Report

## Load the real and synthetic Wisconsin Breast Cancer Datasets

In [2]:
df_real = {}
df_real["train"] = pd.read_csv("../data/WBCD_train.csv")
df_real["test"] = pd.read_csv("../data/WBCD_test.csv")
df_real["train"].shape

(455, 10)

### Choose the synthetic dataset

In [3]:
df_synth = {}
df_synth["train"] = pd.read_csv("../results/data/2024-02-15_Synthpop_455samples.csv")
df_synth["test"] = pd.read_csv("../results/data/2024-02-15_Synthpop_228samples.csv")
df_synth["2nd_gen"] = pd.read_csv(
    "../results/data/2024-02-15_Synthpop_455samples_2nd_gen.csv"
)
df_synth["test"].shape

(228, 10)

## Configure the metadata dictionary

### The continuous and categorical variables need to be specified, as well as the variable to predict

In [4]:
metadata = {
    "continuous": [
        "Clump_Thickness",
        "Uniformity_of_Cell_Size",
        "Uniformity_of_Cell_Shape",
        "Marginal_Adhesion",
        "Single_Epithelial_Cell_Size",
        "Bland_Chromatin",
        "Normal_Nucleoli",
        "Mitoses",
        "Bare_Nuclei",
    ],
    "categorical": ["Class"],
    "variable_to_predict": "Class",
}

## Generate the report

In [5]:
parameters = {  # see the notebooks utility_report and privacy_report for more details
    "cross_learning": False,
    "num_repeat": 1,
    "num_kfolds": 3,
    "num_optuna_trials": 15,
    "use_gpu": True,
    "sampling_frac": 0.5,
}

In [6]:
report = Report(
    dataset_name="Wisconsin Breast Cancer Dataset",
    df_real=df_real,
    df_synthetic=df_synth,
    metadata=metadata,
    figsize=(8, 6),  # will be automatically adjusted for larger or longer figures
    random_state=42,  # for reproducibility purposes
    report_folderpath=None,  # load computed utility and/or privacy reports if available
    report_filename=None,  # the name of the computed report (without extension nor utility/privacy) if available
    metrics=None,  # list of the metrics to compute. Can be utility or privacy metrics. If not specified, all the metrics are computed.
    params=parameters,  # the dictionary containing the parameters for both utility and privacy reports
)

In [7]:
report.compute()

LOGAN test set shape: (228, 10)
TableGan test set shape: (228, 10)
Detector test set shape: (228, 10)


## Get the summary report as a pandas dataframe

In [8]:
report.specification()

----- Wisconsin Breast Cancer Dataset -----
Contains:
    - 455 instances in the train set,
    - 228 instances in the test set,
    - 10 variables, 9 continuous and 1 categorical.


In [9]:
df_summary = report.summary()

In [10]:
by = ["name", "objective", "min", "max"]
df_summary.groupby(by).apply(lambda x: x.drop(by, axis=1).reset_index(drop=True))

alias  \
name                                          objective min max                            
Categorical Consistency                       max       0   1.0 0             cat_consis   
Categorical Statistics                        max       0   1.0 0              cat_stats   
                                                                1              cat_stats   
Classification                                min       0   1.0 0                classif   
Collision                                     max       0   1.0 0              collision   
                                                                1              collision   
                                                                2              collision   
                                                                3              collision   
                                                            inf 0              collision   
                                                                1              collision   
                                                                2              collision   
                                                                3              collision   
                                                                4              collision   
Continuous Consistency                        max       0   1.0 0            cont_consis   
Continuous Statistics                         min       0   inf 0             cont_stats   
                                                                1             cont_stats   
DCR                                           max       0   1.0 0                    dcr   
                                                                1                    dcr   
                                                            inf 0                    dcr   
                                                                1                    dcr   
                                              min       0   1.0 0                    dcr   
                                                                1                    dcr   
Detector                                      max       0   1.0 0               detector   
                                                                1               detector   
                                                                2               detector   
                                                                3               detector   
                                                                4               detector   
Distinguishability                            min       0   1.0 0                   dist   
                                                                1                   dist   
                                                                2                   dist   
                                                                3                   dist   
FScore                                        min       0   inf 0                 fscore   
Feature Importance                            min       0   inf 0            feature_imp   
GAN-Leaks                                     max       0   1.0 0               ganleaks   
                                                                1               ganleaks   
Hellinger Categorical Univariate Distance     min       0   1.0 0     hell_cat_univ_dist   
Hellinger Continuous Univariate Distance      min       0   1.0 0    hell_cont_univ_dist   
KL Divergence Categorical Univariate Distance min       0   inf 0   kl_div_cat_univ_dist   
KL Divergence Continuous Univariate Distance  min       0   inf 0  kl_div_cont_univ_dist   
LOGAN                                         max       0   1.0 0                  logan   
                                                                1                  logan   
                                                                2                  logan   
                                   

## Save and load the report

In [ ]:
with tempfile.TemporaryDirectory() as temp_dir:
    report.save(savepath=temp_dir, filename="report")  # save
    new_report = Report(report_folderpath=temp_dir, report_filename="report")  # load